In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

class LiquidStateMachine(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size):
        super(LiquidStateMachine, self).__init__()

        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size

        self.input_weights = nn.Parameter(torch.FloatTensor(self.reservoir_size, self.input_size))
        self.reservoir_weights = nn.Parameter(torch.FloatTensor(self.reservoir_size, self.reservoir_size))
        self.output_weights = nn.Parameter(torch.FloatTensor(self.output_size, self.reservoir_size))

        self.tanh = nn.Tanh()

        self.reset_parameters()

    def reset_parameters(self):
        #want to initialize to this so we can make sure our weights start in a tanh-like way
        nn.init.xavier_uniform_(self.input_weights)
        nn.init.xavier_uniform_(self.reservoir_weights)
        nn.init.xavier_uniform_(self.output_weights)

    def forward(self, input_data):
        reservoir_state = torch.zeros((self.reservoir_size, 1), dtype=torch.float32)

        for input_sample in input_data:
            input_sample = input_sample.view(-1, 1) # reshape from [sequence_length,] to [sequence_length, 1]
            reservoir_state = self.tanh(
              torch.matmul(self.input_weights, input_sample) +
              torch.matmul(self.reservoir_weights, reservoir_state))

        output = torch.matmul(self.output_weights, reservoir_state)
        return output.squeeze()


In [10]:
def generate_mackey_glass(T, beta=0.2, gamma=0.1, n=0.1, tau=17):
    x = np.zeros(T)
    x[0] = 1.5

    for t in range(T - 1):
        x[t + 1] = x[t] + (beta * x[t - tau]) / (1 + x[t - tau]**10) - gamma * x[t]
        x[t + 1] *= (1 - n)

    return x

T = 2000
mackey_glass_data = generate_mackey_glass(T)

In [11]:
#No strong need to save and load, just wanted to practice it
np.savetxt('mackey_glass.csv', mackey_glass_data, delimiter=',')
data = np.loadtxt('mackey_glass.csv', delimiter=',')

In [12]:
sequence_length = 10
input_data = np.array([data[i:i+sequence_length] for i in range(len(data)-sequence_length)])
target_data = data[sequence_length:]

In [13]:
input_data = (input_data - np.mean(input_data)) / np.std(input_data)

input_train, input_test, target_train, target_test = train_test_split(input_data, target_data, test_size=0.2, shuffle=False)

In [14]:
input_train = torch.from_numpy(input_train).float()
input_test = torch.from_numpy(input_test).float()
target_train = torch.from_numpy(target_train).float()
target_test = torch.from_numpy(target_test).float()

In [15]:
input_size = sequence_length
reservoir_size = 100
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [16]:
model = LiquidStateMachine(input_size, reservoir_size, output_size)

In [17]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
for epoch in range(num_epochs):
    output_train = model(input_train)
    loss = criterion(output_train, target_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1592])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 0.0427
Epoch [20/100], Loss: 0.0113
Epoch [30/100], Loss: 0.0073
Epoch [40/100], Loss: 0.0040
Epoch [50/100], Loss: 0.0035
Epoch [60/100], Loss: 0.0034
Epoch [70/100], Loss: 0.0031
Epoch [80/100], Loss: 0.0031
Epoch [90/100], Loss: 0.0031
Epoch [100/100], Loss: 0.0031


In [19]:
model.eval()
with torch.no_grad():
    output_test = model(input_test)
    test_loss = criterion(output_test, target_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.0023


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([398])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
